In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import requests
import os
import json
import zipfile

## Data Collection

### Files sources

In [ ]:
# URLs of the files
train_datas_url = 'https://www.raphaelcousin.com/modules/data-science-practice/module4/exercise/module4_exercise_train.zip'
test_data_url = 'https://www.raphaelcousin.com/modules/data-science-practice/module4/exercise/Neighborhood_Market_data.csv'

# Function to download a file
def download_file(url, file_name):
    response = requests.get(url)
    response.raise_for_status()  # Ensure we notice bad responses
    with open(file_name, 'wb') as file:
        file.write(response.content)
    print(f'Downloaded {file_name} from {url}')

# Downloading the files
download_file(train_datas_url, 'module4_exercise_train.zip')
download_file(test_data_url, 'Neighborhood_Market_data.csv')

In [ ]:
with zipfile.ZipFile('module4_exercise_train.zip', 'r') as zip_ref:
    zip_ref.extractall('extracted_data')
    print("ZIP文件已解压到 'extracted_data' 文件夹")

#### CityMart

In [ ]:
# read "CityMart_data.csv"
citymart_path = os.path.join('extracted_data', 'CityMart_data.csv')
CityMart = pd.read_csv(citymart_path, sep=',',index_col='item_code')
CityMart

#### Greenfield_Grocers

In [ ]:
# read "Greenfield_Grocers_data.csv"
greenfield_path = os.path.join('extracted_data', 'Greenfield_Grocers_data.csv')
Greenfield_Grocers = pd.read_csv(greenfield_path,sep='|',header=3,index_col='ITEM_CODE')
Greenfield_Grocers

In [ ]:
Greenfield_Grocers.drop('Unnamed: 12' , axis=1, inplace=True)
Greenfield_Grocers.drop('1', axis=1, inplace=True)
Greenfield_Grocers.columns = [col.lower() for col in Greenfield_Grocers.columns]
Greenfield_Grocers.rename_axis(Greenfield_Grocers.index.name.lower(), inplace=True)
Greenfield_Grocers

#### Outlet_data

In [ ]:
# read "SuperSaver_Outlet_data.xlsx"
greenfield_path = os.path.join('extracted_data', 'SuperSaver_Outlet_data.xlsx')
df1 = pd.read_excel(greenfield_path,sheet_name='Quantity')
df2 = pd.read_excel(greenfield_path,sheet_name='Info')
df2.columns = df2.columns.str.strip().str.replace(' ', '_')
column_names = [
    'item_code', 'store_name', 'mass',
    'dimension_length', 'dimension_width', 'dimension_height',
    'days_since_last_purchase', 'package_volume','stock_age'
]
df2 = df2.iloc[:, :len(column_names)]
df2.columns = column_names
df2

In [ ]:
SuperSaver_Outlet_data = pd.merge(df1, df2, on='item_code')
SuperSaver_Outlet_data = SuperSaver_Outlet_data[['item_code', 'store_name', 'mass', 'dimension_length',
                'dimension_width', 'dimension_height', 'days_since_last_purchase',
                'package_volume', 'quantity_sold']]
SuperSaver_Outlet_data.set_index('item_code', inplace=True)
SuperSaver_Outlet_data

#### HighStreet_Bazaar

In [ ]:
# read 'HighStreet_Bazaar_data.json'
greenfield_path = os.path.join('extracted_data', 'HighStreet_Bazaar_data.json')
HighStreet_Bazaar_data = pd.read_json(greenfield_path, orient='records')
HighStreet_Bazaar_data.set_index('item_code', inplace=True)
HighStreet_Bazaar_data

In [ ]:
data = pd.concat([CityMart, Greenfield_Grocers, SuperSaver_Outlet_data, HighStreet_Bazaar_data])
data

#### Aggregate

#### Simple baseline

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import numpy as np

def get_simple_baseline(data, fillna_value=-1, drop_cols=None, k_fold=5, scaler='standard', model='linear', metric='mae', target_col=None, X_data_test=None):

    data = data.copy()
    # Handle missing values
    data.fillna(fillna_value, inplace=True)
    if X_data_test is not None:
        X_data_test = X_data_test.copy()
        X_data_test.fillna(fillna_value, inplace=True)

    # Drop unwanted columns
    if drop_cols:
        data.drop(drop_cols, axis=1, inplace=True)
        if X_data_test is not None:
            X_data_test.drop(drop_cols, axis=1, inplace=True)

    # Split data into features (X) and target (y)
    y = data[target_col]
    X = data.drop(target_col, axis=1)

    # Feature scaling
    if scaler == 'standard':
        scaler = StandardScaler()
    elif scaler == 'minmax':
        scaler = MinMaxScaler()
    else:
        scaler = None

    if scaler:
        X = scaler.fit_transform(X)
        if X_data_test is not None:
            X_data_test = scaler.transform(X_data_test)

    # Initialize the model
    if model == 'linear':
        model = LinearRegression()
    elif model == 'logistic':
        model = LogisticRegression()
    elif model == 'random_forest':
        model = RandomForestClassifier()
    else:
        raise ValueError("Unsupported model type")

    # Initialize cross-validation
    kf = KFold(n_splits=k_fold, shuffle=True, random_state=42)
    scores = []

    # Train and evaluate using k-fold cross-validation
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Train the model
        model.fit(X_train, y_train)

        # Make predictions
        y_pred = model.predict(X_test)

        # Evaluate using the specified metric
        if metric == 'mae':
            score = mean_absolute_error(y_test, y_pred)
        elif metric == 'accuracy':
            score = accuracy_score(y_test, np.round(y_pred))
        else:
            raise ValueError("Unsupported metric")

        scores.append(score)

    if X_data_test is not None:
        model.fit(X, y)
        return np.mean(scores), model.predict(X_data_test)

    # Return the average score
    return np.mean(scores)

In [ ]:
# get_simple_baseline(data, fillna_value=-1, drop_cols=['store_name', 'last_modified'], k_fold=5, scaler='standard', model='linear', metric='mae', target_col='quantity_sold')
get_simple_baseline(data, fillna_value=-1, drop_cols=['store_name', 'last_modified'], k_fold=5, scaler='standard', model='linear', metric='mae', target_col='quantity_sold')

### API sources

In [ ]:
def get_api(endpoint_url):
    try:
        # Make the GET request to the mock API
        response = requests.get(endpoint_url)

        if response.status_code == 200:
            data = response.json()
            print(data["message"])
            return data['data']
        else:
            print(f"Failed to retrieve volume data. Status code: {response.status_code}")

    except Exception as e:
        print(f"An error occurred: {e}")
# password =
password = get_api("https://www.raphaelcousin.com/api/exercise/auth")["password"]
print(password)
# prices =
prices = get_api(f"https://www.raphaelcousin.com/api/exercise/{password}/prices")
prices

In [ ]:
# df_prices =
df_prices = pd.DataFrame.from_dict(prices,orient = 'index', columns=['prices'])
df_prices

#### Aggregate

In [ ]:
# data = pd.merge(data, df_prices, left_index=True, right_index=True, how='left')
# data
data = pd.merge(data, df_prices, left_index=True, right_index=True, how='left')
data

In [ ]:
get_simple_baseline(data, fillna_value=-1, drop_cols=['store_name', 'last_modified'], k_fold=5, scaler='standard', model='linear', metric='mae', target_col='quantity_sold')

### Scrapping sources

In [ ]:
!pip install selenium

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set up the Selenium WebDriver (e.g., Chrome)
driver = webdriver.Chrome()  # Make sure ChromeDriver is installed
# driver = webdriver.Firefox()
# driver = webdriver.Edge()
# driver = webdriver.Safari()

# Open the URL
url = 'https://www.raphaelcousin.com/module4/scrapable-data'
driver.get(url)

# Wait for the page to fully load (increase time if needed)
time.sleep(5)

# Get the fully rendered page source
html = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Initialize lists to store scraped data
exercise_data = []

# Find both tables
tables = soup.find_all('table')

# Close the Selenium WebDriver
driver.quit()

# Scrape the second table (Exercise Data)
course_table = tables[1]
for row in course_table.find('tbody').find_all('tr'):
    cols = row.find_all('td')
    exercise_data.append({
        'Item code': cols[0].text,
        'Customer score': float(cols[1].text),
        'Total Reviews': cols[2].text
     })

# Convert the lists to pandas DataFrames
df_exercise = pd.DataFrame(exercise_data)
df_exercise.columns = [col.lower() for col in df_exercise.columns]
df_exercise.columns = df_exercise.columns.str.replace(' ', '_')
df_exercise.set_index('item_code', inplace=True)
df_exercise



#### Aggregate

In [ ]:
data = pd.merge(data, df_exercise, left_index=True, right_index=True, how='left')
data

In [ ]:
get_simple_baseline(data, fillna_value=-1, drop_cols=['store_name', 'last_modified'], k_fold=5, scaler='standard', model='linear', metric='mae', target_col='quantity_sold')

### Generating Submission File

In [ ]:
# X_test =  read  Neighborhood_Market_data

# read
df_StoreN =  pd.read_csv("Neighborhood_Market_data.csv", sep=",", index_col='item_code')
df_StoreN

In [ ]:
df_StoreN = pd.merge(df_StoreN, df_prices, left_index=True, right_index=True, how='left')
df_StoreN = pd.merge(df_StoreN, df_exercise, left_index=True, right_index=True, how='left')

In [ ]:
 _, x_pred = get_simple_baseline(data, fillna_value=-1, drop_cols=['store_name', 'last_modified'], k_fold=5, scaler='standard', model='linear', metric='mae', target_col='quantity_sold', X_data_test = df_StoreN)

In [ ]:
submission = pd.DataFrame({
    'item_code': df_StoreN.index,
    'quantity_sold': x_pred # your_prediction
})

submission.to_csv('submission.csv', index=False, sep=',')
submission.head()